In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.7 MB/s eta 0:00:00


In [3]:
import sqlite3
import random
from faker import Faker

# --- Configuration ---
DATABASE_NAME = 'OnlineBookRetailer.sqlite'
NUM_CUSTOMERS = 150
NUM_PRODUCTS = 50
NUM_ORDERS = 1200 # Main table with 1000+ rows

fake = Faker('en_GB') # Use UK locale for more realistic data (like postcodes)

# --- Data Type Mapping (Required for Assignment) ---
# Nominal: customer_country, product_category
# Ordinal: product_rating, order_status
# Interval: order_timestamp (UNIX timestamp - arbitrary zero)
# Ratio: product_price, order_total_amount, order_quantity (meaningful zero)


def create_tables(conn):
    """Creates the Customer, Product, and Orders tables with constraints."""
    cursor = conn.cursor()

    # 1. Customers Table (Primary Key, Nominal Data)
    # Constraints: customer_email is UNIQUE and NOT NULL
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Customers (
            customer_id INTEGER PRIMARY KEY,
            customer_name TEXT NOT NULL,
            customer_email TEXT UNIQUE NOT NULL,
            customer_country TEXT NOT NULL, -- Nominal Data
            customer_signup_date TEXT NOT NULL
        );
    """)

    # 2. Products Table (Primary Key, Composite Key, Nominal/Ratio Data)
    # Constraint: product_sku is a UNIQUE key (part of the composite key here)
    # Composite Key: (product_name, product_sku)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Products (
            product_id INTEGER PRIMARY KEY,
            product_name TEXT NOT NULL,
            product_sku TEXT NOT NULL,
            product_category TEXT NOT NULL, -- Nominal Data
            product_price REAL NOT NULL,    -- Ratio Data (price, meaningful zero)
            product_rating INTEGER,         -- Ordinal Data (1-5 star rating)
            UNIQUE(product_name, product_sku)
        );
    """)

    # 3. Orders Table (Primary/Foreign Keys, Ordinal/Interval/Ratio Data)
    # Constraints: Foreign Keys linking to Customers and Products.
    # Checks: order_quantity > 0
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Orders (
            order_id INTEGER PRIMARY KEY,
            customer_id INTEGER,
            product_id INTEGER,
            order_timestamp INTEGER NOT NULL,  -- Interval Data (UNIX time)
            order_quantity INTEGER NOT NULL CHECK(order_quantity > 0), -- Ratio Data
            order_total_amount REAL NOT NULL, -- Ratio Data
            order_status TEXT NOT NULL,       -- Ordinal Data (e.g., Shipped, Delivered)
            FOREIGN KEY (customer_id) REFERENCES Customers(customer_id) ON DELETE CASCADE,
            FOREIGN KEY (product_id) REFERENCES Products(product_id) ON DELETE RESTRICT
        );
    """)
    conn.commit()


def generate_data(conn):
    """Generates and inserts realistic, randomised data."""
    cursor = conn.cursor()

    # --- 1. Generate Customers Data ---
    print(f"Generating {NUM_CUSTOMERS} customers...")
    customer_data = []
    # Deliberately introduce a duplicate email:
    duplicate_email = fake.unique.email()
    for i in range(NUM_CUSTOMERS):
        if i == NUM_CUSTOMERS - 1:
            email = duplicate_email # Insertion will fail/be ignored if UNIQUE constraint is on
        elif i == 0:
            email = duplicate_email # Insert it here first, so the last one fails.
        else:
            email = fake.unique.email()

        customer_data.append((
            fake.name(),
            email,
            random.choice(['USA', 'UK', 'Canada', 'Australia', 'Germany', 'France']),
            fake.date_this_decade()
        ))

    # Using 'OR IGNORE' handles the intentional duplicate email gracefully
    cursor.executemany("""
        INSERT OR IGNORE INTO Customers (customer_name, customer_email, customer_country, customer_signup_date)
        VALUES (?, ?, ?, ?)
    """, customer_data)
    conn.commit()
    print(f"Customers generated. Actual count: {cursor.execute('SELECT COUNT(*) FROM Customers').fetchone()[0]}")

    # --- 2. Generate Products Data ---
    print(f"Generating {NUM_PRODUCTS} products...")
    product_data = []
    product_categories = ['Fiction', 'Non-Fiction', 'Biography', 'Science', 'Mystery', 'Fantasy', 'Programming']
    for i in range(NUM_PRODUCTS):
        product_name = fake.bs() + " Book"
        product_sku = f"SKU-{random.randint(10000, 99999)}"
        product_category = random.choice(product_categories)
        product_price = round(random.uniform(5.99, 49.99), 2)

        # Introduce missing data (rating)
        product_rating = random.choice([None, random.randint(1, 5)])

        product_data.append((
            product_name,
            product_sku,
            product_category,
            product_price,
            product_rating
        ))

    cursor.executemany("""
        INSERT INTO Products (product_name, product_sku, product_category, product_price, product_rating)
        VALUES (?, ?, ?, ?, ?)
    """, product_data)
    conn.commit()

    # --- 3. Generate Orders Data ---
    print(f"Generating {NUM_ORDERS} orders (1000+ rows)...")

    # Get valid customer and product IDs
    customer_ids = [row[0] for row in cursor.execute("SELECT customer_id FROM Customers").fetchall()]
    product_ids = [row[0] for row in cursor.execute("SELECT product_id FROM Products").fetchall()]

    order_data = []
    order_statuses = ['Pending', 'Shipped', 'Delivered', 'Cancelled']

    # Get product prices for total calculation
    product_prices = {row[0]: row[1] for row in cursor.execute("SELECT product_id, product_price FROM Products").fetchall()}

    for i in range(NUM_ORDERS):
        customer_id = random.choice(customer_ids)
        product_id = random.choice(product_ids)

        # Interval Data: UNIX timestamp
        order_timestamp = fake.date_time_this_year().timestamp()

        # Ratio Data: Quantity (ratio, meaningful zero - enforced > 0 by CHECK)
        order_quantity = random.randint(1, 5)

        # Ratio Data: Total Amount
        unit_price = product_prices.get(product_id, 10.0) # Default if lookup fails
        order_total_amount = round(unit_price * order_quantity * random.uniform(0.95, 1.05), 2) # Adding a small random variation/discount

        # Ordinal Data: Order Status (Ordered, Shipped, Delivered, etc.)
        order_status = random.choice(order_statuses)

        order_data.append((
            customer_id,
            product_id,
            int(order_timestamp),
            order_quantity,
            order_total_amount,
            order_status
        ))

    # Deliberately insert a duplicate order for realism (order_id will be different but other values are the same)
    order_data.append(order_data[random.randint(0, NUM_ORDERS - 1)])

    cursor.executemany("""
        INSERT INTO Orders (customer_id, product_id, order_timestamp, order_quantity, order_total_amount, order_status)
        VALUES (?, ?, ?, ?, ?, ?)
    """, order_data)
    conn.commit()
    print(f"Orders generated. Actual count: {cursor.execute('SELECT COUNT(*) FROM Orders').fetchone()[0]}")


# --- Main Execution ---
if __name__ == '__main__':
    print(f"Attempting to create database: {DATABASE_NAME}")
    try:
        # Connect to SQLite database (creates the file if it doesn't exist)
        conn = sqlite3.connect(DATABASE_NAME)

        # Enable foreign key enforcement for better realism
        conn.execute("PRAGMA foreign_keys = ON;")

        create_tables(conn)
        generate_data(conn)

        conn.close()
        print(f"\nDatabase '{DATABASE_NAME}' created successfully with all required data.")
        print("You can find the .sqlite file in the same directory as this script.")

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

Attempting to create database: OnlineBookRetailer.sqlite
Generating 150 customers...
Customers generated. Actual count: 149
Generating 50 products...
Generating 1200 orders (1000+ rows)...
Orders generated. Actual count: 1201

Database 'OnlineBookRetailer.sqlite' created successfully with all required data.
You can find the .sqlite file in the same directory as this script.


/tmp/ipython-input-3873454219.py:95: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.executemany("""
